# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [1]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os

import pandas as pd
from google.cloud import bigquery
 

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="infinite-blade-452909-r4-d980a9c88b70.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [3]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df


C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AR,2021-01-30,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
1,AR,2022-05-26,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
2,AR,2021-06-11,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
3,AR,2022-04-24,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
4,AR,2020-04-06,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
5,AR,2021-10-19,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
6,AR,2022-03-24,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
7,AR,2022-04-28,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
8,AR,2022-02-28,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
9,AR,2020-10-07,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)


# Część 3
Misja dodatkowa - Poznanie danych

### 3.1 Ile jest zapisanych wierszy z danymi.

In [4]:
dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

### 3.2 Ile krajów jest uwzględnionych w danych.

In [5]:
query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


np.int64(246)

 ### 3.3. W jaki sposób zapisywane są dzienne informacje dla krajów.

In [6]:
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [7]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


np.True_

### 3.4. W jaki sposób zapisywane są wartości liczbowe.

In [8]:
country_all.dtypes.value_counts()

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [9]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [10]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Jaki przedział czasowy jest uwzględniony w danych. Jakie są przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób.

In [11]:
query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [12]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


In [13]:
# Dla krajów z osobna
query = """
SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_confirmed' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_confirmed IS NOT NULL AND new_confirmed > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_deceased' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_deceased IS NOT NULL AND new_deceased > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_persons_vaccinated' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated > 0 AND aggregation_level=0
GROUP BY country_name
ORDER BY country_name, metric
"""

dates_info = client.query(query).to_dataframe()
dates_info

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,country_name,start_date,end_date,metric
0,Afghanistan,2020-02-22,2022-09-13,new_confirmed
1,Afghanistan,2020-03-21,2022-09-12,new_deceased
2,Afghanistan,2021-02-28,2022-09-12,new_persons_vaccinated
3,Albania,2020-03-07,2022-09-13,new_confirmed
4,Albania,2020-03-09,2022-09-11,new_deceased
...,...,...,...,...
663,Zambia,2020-01-02,2022-09-10,new_deceased
664,Zambia,2021-04-15,2022-08-07,new_persons_vaccinated
665,Zimbabwe,2020-01-02,2022-09-12,new_confirmed
666,Zimbabwe,2020-01-02,2022-08-29,new_deceased


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [14]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [15]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [16]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [17]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [18]:
query = ("""
SELECT DISTINCT date, country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
ORDER BY country_name
LIMIT 6
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,Country_name,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2022-09-14,Afghanistan,AFG,<NA>,<NA>,<NA>,<NA>
1,2022-06-05,Afghanistan,AFG,0,2,1,0
2,2020-10-25,Afghanistan,AFG,1,2,0,0
3,2021-06-13,Afghanistan,AFG,3,2,2,0
4,2021-05-25,Afghanistan,AFG,0,3,2,0
5,2020-06-14,Afghanistan,AFG,3,3,2,4


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


### 4.1 Podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [19]:
query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd
0,Netherlands Antilles,None,197041,800,NaN,<NA>,<NA>
1,Aruba,ABW,106766,180,593.144,46295,29007
2,Bangladesh,BGD,164689383,147630,1265.187,60987417,1855
3,Botswana,BWA,2351625,581730,4.149,1616550,7961
4,China,CHN,1439323774,9562910,153.312,842933962,10261
...,...,...,...,...,...,...,...
241,New Zealand,NZL,4822233,267710,18.314,4258860,42084
242,Sierra Leone,SLE,7976985,72300,110.515,3319366,504
243,Turkmenistan,TKM,6031187,488100,12.834,3092738,6966
244,Vatican City,VAT,809,0,1838.636,<NA>,<NA>


### 4.2 statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [20]:
# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_confirmed_cases
0,2020-01-01,GNQ,0
1,2020-01-01,CYP,0
2,2020-01-01,LBR,0
3,2020-01-01,LBY,0
4,2020-01-01,MWI,0
...,...,...,...
243235,2022-09-15,AUS,1796
243254,2022-09-15,BRA,9931
243259,2022-09-15,TWN,45405
243273,2022-09-15,MYS,2375


### 4.3 Uwypuklenie problemu śmiertelności.

In [21]:
query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_new_deceased
0,2021-01-30,ARG,167
1,2021-02-03,CZE,137
2,2020-10-15,DEU,91
3,2021-12-14,FRA,158
4,2021-08-15,FRA,44
...,...,...,...
243781,2022-07-28,GTM,15
243782,2022-07-24,SWE,15
243783,2021-08-26,VEN,15
243784,2020-06-07,ROU,15


### 4.4 Trendy i zależności dotyczące szczepień na COVID-19.

In [22]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vacinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vacinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_new_vacinations,New_persons_fully_vaccinated
0,2022-04-28,ARG,1335,6105
1,2021-03-23,EST,2745,325
2,2021-12-12,FIN,0,24057
3,2021-12-14,FRA,27950,35820
4,2021-12-13,PER,114466,131995
...,...,...,...,...
243781,2021-05-09,ARE,0,0
243782,2021-03-10,ARE,0,0
243783,2022-07-25,AUT,87,0
243784,2022-03-04,AUT,924,0


### 4.5 Liczba wykonanych testów

In [23]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)

display(investment_in_vaccines)

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_tested
66,2020-01-01,ARG,4
105,2020-01-01,BRA,1154
142,2020-01-01,MEX,72
331,2020-01-02,BRA,2337
346,2020-01-02,ARG,42
...,...,...,...
243137,2022-09-15,MYS,0
243200,2022-09-15,AUT,67392
243212,2022-09-15,ITA,146983
243253,2022-09-15,AUS,4367


# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [24]:
df_data_from_4 = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_trend_analysis, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, investment_in_vaccines, on=['date', '3_letter_code'])
df_data_from_4.to_csv("..\wyniki\\data_from_4.csv", index=False)
df_data_from_4

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\4073914796.py:4: SyntaxWarning: invalid escape sequence '\w'
  df_data_from_4.to_csv("..\wyniki\\data_from_4.csv", index=False)


,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Aruba,ABW,106766,180,593.144,46295,29007,2021-06-28,1,179,406,365
1,Aruba,ABW,106766,180,593.144,46295,29007,2021-06-29,3,153,359,428
2,Bangladesh,BGD,164689383,147630,1265.187,60987417,1855,2021-01-27,454,26,<NA>,14830
3,Bangladesh,BGD,164689383,147630,1265.187,60987417,1855,2021-01-28,363,541,<NA>,12075
4,Bangladesh,BGD,164689383,147630,1265.187,60987417,1855,2021-02-07,387,30593,<NA>,13762
...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Estonia,EST,1324820,45340,30.400,916024,23246,2022-09-08,131,37,37,991
31514,Estonia,EST,1324820,45340,30.400,916024,23246,2022-09-09,144,73,70,969
31515,Estonia,EST,1324820,45340,30.400,916024,23246,2022-09-10,92,16,17,611
31516,Estonia,EST,1324820,45340,30.400,916024,23246,2022-09-11,46,13,14,619


# Część 6


### 6.1 Usunięcie powtarzających się kolumn, usuwamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [25]:
df_data_from_4.drop('population', axis=1, inplace=True)
df_data_from_4.drop('area_sq_km', axis=1, inplace=True)
df_data_from_4.drop('population_density', axis=1, inplace=True)
df_data_from_4.drop('gdp_per_capita_usd', axis=1, inplace=True)

df_data_from_4

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Aruba,ABW,46295,2021-06-28,1,179,406,365
1,Aruba,ABW,46295,2021-06-29,3,153,359,428
2,Bangladesh,BGD,60987417,2021-01-27,454,26,<NA>,14830
3,Bangladesh,BGD,60987417,2021-01-28,363,541,<NA>,12075
4,Bangladesh,BGD,60987417,2021-02-07,387,30593,<NA>,13762
...,...,...,...,...,...,...,...,...
31513,Estonia,EST,916024,2022-09-08,131,37,37,991
31514,Estonia,EST,916024,2022-09-09,144,73,70,969
31515,Estonia,EST,916024,2022-09-10,92,16,17,611
31516,Estonia,EST,916024,2022-09-11,46,13,14,619


### 6.2 Podstawowe infromacje geograficzne i demograficzne o krajach

In [26]:
# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1

world_countries = pd.read_csv("world_countries.csv")

df_with_countries = pd.merge(df_data_from_4, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
df_with_countries.drop('Country/Territory', axis=1, inplace=True)
df_with_countries.drop('CCA3', axis=1, inplace=True)

df_with_countries

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,Aruba,ABW,46295,2021-06-28,1,179,406,365,198,Oranjestad,...,104257,100341,89101,65712,62267,59106,180,591.3611,0.9991,0.00
1,Aruba,ABW,46295,2021-06-29,3,153,359,428,198,Oranjestad,...,104257,100341,89101,65712,62267,59106,180,591.3611,0.9991,0.00
2,Bangladesh,BGD,60987417,2021-01-27,454,26,<NA>,14830,8,Dhaka,...,157830000,148391139,129193327,107147651,83929765,67541860,147570,1160.0350,1.0108,2.15
3,Bangladesh,BGD,60987417,2021-01-28,363,541,<NA>,12075,8,Dhaka,...,157830000,148391139,129193327,107147651,83929765,67541860,147570,1160.0350,1.0108,2.15
4,Bangladesh,BGD,60987417,2021-02-07,387,30593,<NA>,13762,8,Dhaka,...,157830000,148391139,129193327,107147651,83929765,67541860,147570,1160.0350,1.0108,2.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Estonia,EST,916024,2022-09-08,131,37,37,991,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.9980,0.02
31514,Estonia,EST,916024,2022-09-09,144,73,70,969,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.9980,0.02
31515,Estonia,EST,916024,2022-09-10,92,16,17,611,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.9980,0.02
31516,Estonia,EST,916024,2022-09-11,46,13,14,619,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.9980,0.02


In [27]:
df_gdp = pd.read_csv("gdp.csv")
df_gdp_pivot = df_gdp.pivot(index="Country Code", columns="Year", values="Value")
df_gdp_pivot.columns = [f"GDP_{int(year)}" for year in df_gdp_pivot.columns]
df_gdp_pivot.reset_index(inplace=True)
df_with_gdp = df_with_countries.merge(df_gdp_pivot, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_gdp.drop(columns=["Country Code"], inplace=True)
display(df_with_gdp)

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,GDP_2007,GDP_2008,GDP_2009,GDP_2010,GDP_2011,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016
0,Aruba,ABW,46295,2021-06-28,1,179,406,365,198,Oranjestad,...,2.623726e+09,2.791961e+09,2.498933e+09,2.467704e+09,2.584464e+09,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,46295,2021-06-29,3,153,359,428,198,Oranjestad,...,2.623726e+09,2.791961e+09,2.498933e+09,2.467704e+09,2.584464e+09,NaN,NaN,NaN,NaN,NaN
2,Bangladesh,BGD,60987417,2021-01-27,454,26,<NA>,14830,8,Dhaka,...,7.961189e+10,9.163128e+10,1.024778e+11,1.152791e+11,1.286379e+11,1.333557e+11,1.499905e+11,1.728855e+11,1.950787e+11,2.214152e+11
3,Bangladesh,BGD,60987417,2021-01-28,363,541,<NA>,12075,8,Dhaka,...,7.961189e+10,9.163128e+10,1.024778e+11,1.152791e+11,1.286379e+11,1.333557e+11,1.499905e+11,1.728855e+11,1.950787e+11,2.214152e+11
4,Bangladesh,BGD,60987417,2021-02-07,387,30593,<NA>,13762,8,Dhaka,...,7.961189e+10,9.163128e+10,1.024778e+11,1.152791e+11,1.286379e+11,1.333557e+11,1.499905e+11,1.728855e+11,1.950787e+11,2.214152e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Estonia,EST,916024,2022-09-08,131,37,37,991,156,Tallinn,...,2.223706e+10,2.419404e+10,1.965249e+10,1.949094e+10,2.317024e+10,2.304386e+10,2.513715e+10,2.622462e+10,2.256696e+10,2.333791e+10
31514,Estonia,EST,916024,2022-09-09,144,73,70,969,156,Tallinn,...,2.223706e+10,2.419404e+10,1.965249e+10,1.949094e+10,2.317024e+10,2.304386e+10,2.513715e+10,2.622462e+10,2.256696e+10,2.333791e+10
31515,Estonia,EST,916024,2022-09-10,92,16,17,611,156,Tallinn,...,2.223706e+10,2.419404e+10,1.965249e+10,1.949094e+10,2.317024e+10,2.304386e+10,2.513715e+10,2.622462e+10,2.256696e+10,2.333791e+10
31516,Estonia,EST,916024,2022-09-11,46,13,14,619,156,Tallinn,...,2.223706e+10,2.419404e+10,1.965249e+10,1.949094e+10,2.317024e+10,2.304386e+10,2.513715e+10,2.622462e+10,2.256696e+10,2.333791e+10


# Część 7

### 7.1 Dane związane z inflacją

In [28]:
# https://www.worldbank.org/en/research/brief/inflation-database

# Z wszystkich strony arkusza wybraliśmy ogólny przyrost inflacji w danych latach
df_inflation = pd.read_excel("load_data\\inflation_all.xlsx", sheet_name="hcpi_a")

df_inflation.drop(columns=["IMF Country Code", "Indicator Type", "Series Name", "Note", "Country"], inplace=True)
df_inflation = df_inflation.melt(id_vars=["Country Code"], var_name="Year", value_name="Inflation")
df_inflation.dropna(subset=["Inflation"], inplace=True)
df_inflation = df_inflation.pivot(index="Country Code", columns="Year", values="Inflation")
df_inflation.columns = [f"Inflation_{int(year)}" for year in df_inflation.columns]
df_inflation.reset_index(inplace=True)
df_with_inflation = df_with_gdp.merge(df_inflation, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_inflation.drop(columns=["Country Code"], inplace=True)
display(df_with_inflation)

<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:16: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\907530732.py:16: SyntaxWarning: invalid escape sequence '\w'
  df_with_inflation.to_csv("..\wyniki\\with_gdp_inflation.csv", index=False)


,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,Inflation_2014,Inflation_2015,Inflation_2016,Inflation_2017,Inflation_2018,Inflation_2019,Inflation_2020,Inflation_2021,Inflation_2022,Inflation_2023
0,Aruba,ABW,46295,2021-06-28,1,179,406,365,198,Oranjestad,...,0.420000,0.480000,-0.890000,-0.474,3.58,4.257,1.222000,0.744000,5.520000,3.366000
1,Aruba,ABW,46295,2021-06-29,3,153,359,428,198,Oranjestad,...,0.420000,0.480000,-0.890000,-0.474,3.58,4.257,1.222000,0.744000,5.520000,3.366000
2,Bangladesh,BGD,60987417,2021-01-27,454,26,<NA>,14830,8,Dhaka,...,6.982977,6.185943,5.505753,5.611,5.55,5.475,5.691075,5.306676,7.696954,9.783334
3,Bangladesh,BGD,60987417,2021-01-28,363,541,<NA>,12075,8,Dhaka,...,6.982977,6.185943,5.505753,5.611,5.55,5.475,5.691075,5.306676,7.696954,9.783334
4,Bangladesh,BGD,60987417,2021-02-07,387,30593,<NA>,13762,8,Dhaka,...,6.982977,6.185943,5.505753,5.611,5.55,5.475,5.691075,5.306676,7.696954,9.783334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Estonia,EST,916024,2022-09-08,131,37,37,991,156,Tallinn,...,-0.106180,-0.492323,0.148682,3.651,3.44,2.268,-0.444531,4.653167,19.398260,9.452177
31514,Estonia,EST,916024,2022-09-09,144,73,70,969,156,Tallinn,...,-0.106180,-0.492323,0.148682,3.651,3.44,2.268,-0.444531,4.653167,19.398260,9.452177
31515,Estonia,EST,916024,2022-09-10,92,16,17,611,156,Tallinn,...,-0.106180,-0.492323,0.148682,3.651,3.44,2.268,-0.444531,4.653167,19.398260,9.452177
31516,Estonia,EST,916024,2022-09-11,46,13,14,619,156,Tallinn,...,-0.106180,-0.492323,0.148682,3.651,3.44,2.268,-0.444531,4.653167,19.398260,9.452177


### Dane o bezrobociu

In [32]:
#  https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=map&year=1991

df_unemployment = pd.read_csv("load_data\\total_unemployment.csv")
df_unemployment.drop(columns=["Country Name", "Indicator Name", "Indicator Code", "Unnamed: 68"], inplace=True, errors="ignore")
df_unemployment = df_unemployment.melt(id_vars=["Country Code"], var_name="Year", value_name="Unemployment")
df_unemployment.dropna(subset=["Unemployment"], inplace=True)
df_unemployment = df_unemployment.pivot(index="Country Code", columns="Year", values="Unemployment")
df_unemployment.columns = [f"Unemployment_{int(year)}" for year in df_unemployment.columns]
df_unemployment.reset_index(inplace=True)
df_unemployment = df_with_inflation.merge(df_unemployment, left_on="3_letter_code", right_on="Country Code", how="left")
df_unemployment.drop(columns=["Country Code"], inplace=True)
display(df_unemployment)

<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:12: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\89301662.py:12: SyntaxWarning: invalid escape sequence '\w'
  df_unemployment.to_csv("..\wyniki\\with_gdp_inflation_unemployment.csv", index=False)


,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,Unemployment_2014,Unemployment_2015,Unemployment_2016,Unemployment_2017,Unemployment_2018,Unemployment_2019,Unemployment_2020,Unemployment_2021,Unemployment_2022,Unemployment_2023
0,Aruba,ABW,46295,2021-06-28,1,179,406,365,198,Oranjestad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,46295,2021-06-29,3,153,359,428,198,Oranjestad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bangladesh,BGD,60987417,2021-01-27,454,26,<NA>,14830,8,Dhaka,...,4.416,4.396,4.350,4.372,4.407,4.441,5.436,5.327,4.593,4.454
3,Bangladesh,BGD,60987417,2021-01-28,363,541,<NA>,12075,8,Dhaka,...,4.416,4.396,4.350,4.372,4.407,4.441,5.436,5.327,4.593,4.454
4,Bangladesh,BGD,60987417,2021-02-07,387,30593,<NA>,13762,8,Dhaka,...,4.416,4.396,4.350,4.372,4.407,4.441,5.436,5.327,4.593,4.454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Estonia,EST,916024,2022-09-08,131,37,37,991,156,Tallinn,...,7.352,6.412,6.882,5.807,5.412,4.514,6.956,6.178,5.571,6.378
31514,Estonia,EST,916024,2022-09-09,144,73,70,969,156,Tallinn,...,7.352,6.412,6.882,5.807,5.412,4.514,6.956,6.178,5.571,6.378
31515,Estonia,EST,916024,2022-09-10,92,16,17,611,156,Tallinn,...,7.352,6.412,6.882,5.807,5.412,4.514,6.956,6.178,5.571,6.378
31516,Estonia,EST,916024,2022-09-11,46,13,14,619,156,Tallinn,...,7.352,6.412,6.882,5.807,5.412,4.514,6.956,6.178,5.571,6.378


### Dane o przestępstwach w latach

In [53]:
df_crime = pd.read_csv("load_data\\crime_rate_per_100k.csv")
df_crime = df_crime.iloc[:217]
df_crime.drop(columns=["Series Name", "Series Code", "Country Name"], inplace=True, errors="ignore")
df_crime.columns = df_crime.columns.str.replace(r"\s*\[YR\d+\]", "", regex=True)
df_crime = df_crime.melt(id_vars=["Country Code"], var_name="Year", value_name="Crime Rate")
df_crime["Year"] = df_crime["Year"].astype(int)
df_crime["Crime Rate"] = pd.to_numeric(df_crime["Crime Rate"], errors="coerce")
df_crime.dropna(subset=["Crime Rate"], inplace=True)
df_crime = df_crime.pivot(index="Country Code", columns="Year", values="Crime Rate")
df_crime.columns = [f"Crime_Rate_{int(year)}" for year in df_crime.columns]
df_crime.reset_index(inplace=True)
df_final = df_unemployment.merge(df_crime, left_on="3_letter_code", right_on="Country Code", how="left")
df_final.drop(columns=["Country Code"], inplace=True)
df_final.to_csv("..\wyniki\\final_data.csv",  index=False)
display(df_final)

<>:15: SyntaxWarning: invalid escape sequence '\w'
<>:15: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_7468\428596524.py:15: SyntaxWarning: invalid escape sequence '\w'
  df_final.to_csv("..\wyniki\\final_data.csv",  index=False)


,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,Crime_Rate_2012,Crime_Rate_2013,Crime_Rate_2014,Crime_Rate_2015,Crime_Rate_2016,Crime_Rate_2017,Crime_Rate_2018,Crime_Rate_2019,Crime_Rate_2020,Crime_Rate_2021
0,Aruba,ABW,46295,2021-06-28,1,179,406,365,198,Oranjestad,...,3.917286,5.832037,1.930614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,46295,2021-06-29,3,153,359,428,198,Oranjestad,...,3.917286,5.832037,1.930614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bangladesh,BGD,60987417,2021-01-27,454,26,<NA>,14830,8,Dhaka,...,2.704966,2.852039,2.894308,2.556548,2.247401,2.193531,2.339875,NaN,NaN,NaN
3,Bangladesh,BGD,60987417,2021-01-28,363,541,<NA>,12075,8,Dhaka,...,2.704966,2.852039,2.894308,2.556548,2.247401,2.193531,2.339875,NaN,NaN,NaN
4,Bangladesh,BGD,60987417,2021-02-07,387,30593,<NA>,13762,8,Dhaka,...,2.704966,2.852039,2.894308,2.556548,2.247401,2.193531,2.339875,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Estonia,EST,916024,2022-09-08,131,37,37,991,156,Tallinn,...,4.763050,3.945427,3.118995,3.422947,2.507741,2.201057,2.117767,1.95925,3.159215,1.956798
31514,Estonia,EST,916024,2022-09-09,144,73,70,969,156,Tallinn,...,4.763050,3.945427,3.118995,3.422947,2.507741,2.201057,2.117767,1.95925,3.159215,1.956798
31515,Estonia,EST,916024,2022-09-10,92,16,17,611,156,Tallinn,...,4.763050,3.945427,3.118995,3.422947,2.507741,2.201057,2.117767,1.95925,3.159215,1.956798
31516,Estonia,EST,916024,2022-09-11,46,13,14,619,156,Tallinn,...,4.763050,3.945427,3.118995,3.422947,2.507741,2.201057,2.117767,1.95925,3.159215,1.956798
